In [1]:
import os
import time
import warnings
warnings.simplefilter('ignore')
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit, minimize_scalar
from scipy import signal
from sklearn.metrics import r2_score, mean_squared_error

from FuncLaserScattering import delete_distance, smoothing_apple, skip_distance, calc_diff, image2feature
from FuncLaserScattering import fitting

# 散乱データの結合

In [2]:
# %%time
# # 貯蔵期間ごとになっていたので、それぞれを結合し、一つにまとめる


# allfile = os.listdir()
# csv_list = [csvfile for csvfile in allfile if 'Profile' in csvfile]
# csv_list = [csvfile for csvfile in csv_list if 'all' not in csvfile]
# csv_list.pop(0)

# # プロファイルを結合
# profile_all = pd.DataFrame()
# for csv_path in csv_list:
#     df_temp = pd.read_csv(csv_path, header=0, index_col=0)
#     profile_all = pd.concat([profile_all, df_temp],axis=1)

    
# # プロファイルを保存
# profile_all.to_csv('Profile_all.csv')

In [3]:
%%time
# 新しい解析法で得られたデータで特徴量作成
# 粉質化のみの算出

for storage in ['01','02','03','04','06','07','08','09','10','11','12']:#

    ##### データの読み込み #####
    total_df = pd.read_csv(f'./../data/laser/profile/Profile_{storage}.csv')

    # 距離と輝度（強度）を分割
    distance = total_df.iloc[:,0]
    apple_df = total_df.drop('distance (mm)',axis=1)


    #####解析範囲外の領域を削除#####
    distance_30mm, apple_df_30mm = delete_distance(distance, apple_df)


    #####プロファイルの平滑化#####
    distance_smooth, apple_smooth = smoothing_apple(distance_30mm, apple_df_30mm)


    #####プロファイルの間引き (1mm間隔)#####
    distance_eq, apple_smooth_eq = skip_distance(distance_smooth, apple_smooth)


    #####変化率の取得#####
    res = calc_diff(distance_eq, data_smoothing=apple_smooth_eq)


    #####CurveFitting#####
    eff_df = pd.DataFrame()

    distance_eq = distance_eq[1:]
    apple_smooth_eq = apple_smooth_eq.iloc[1:,:]

    for i in range(apple_smooth_eq.shape[1]): 

        eff_temp = fitting(distance_eq,apple_smooth_eq.iloc[:,i])

        eff_df = pd.concat([eff_df,eff_temp],axis=0)

    eff_df.index = res.columns

    # 結合の都合上転置する
    eff_df = eff_df.T


    #####画像から特徴量を抽出#####
    print(f'Start image analysis of {storage}week!!!')

    image_path = f'./../data/laser/image/week{storage}'
    feature_img = image2feature(image_path)
    feature_img.index = res.columns
    
    print(f'Finish image analysis of {storage}week!!!')


    #####全特徴量の結合および保存#####
    feature_df = pd.concat([eff_df, res, feature_img.T],axis=0).T


    ##### データの保存 #####
    # 平滑化後のプロファイルおよび特徴量を保存
    apple_smooth_eq.to_csv(f'./../data/laser/profile/Profile_{storage}_Smooth.csv')
    feature_df.to_csv(f'./../data/feature/feature_{storage}.csv')
        

Start image analysis of 01week!!!


KeyboardInterrupt: 

In [9]:
%%time
# 貯蔵期間ごとになっていたので、それぞれを結合し、一つにまとめる

feature_path = './../data/feature'
allfile = os.listdir(feature_path)
csv_list = [csvfile for csvfile in allfile if 'feature' in csvfile]
csv_list = [csvfile for csvfile in csv_list if 'all' not in csvfile]
csv_list = [csvfile for csvfile in csv_list if 'laser' not in csvfile]
csv_list = [csvfile for csvfile in csv_list if 'XrayCT' not in csvfile]
csv_list.sort()
# csv_list.pop(0)

print(csv_list)

# プロファイルを結合
feature_all = pd.DataFrame()
for csv_path in csv_list:
    df_temp = pd.read_csv(f'{feature_path}/{csv_path}', header=0, index_col=0)
    feature_all = pd.concat([feature_all, df_temp],axis=0)

    
# プロファイルを保存
feature_all.to_csv(f'{feature_path}/feature_all.csv')

# laser用のfeature.csv作成
temp_index = [x for x in feature_all.index.tolist() if '_1_1' in x or '_2_1' in x or '_3_1' in x]
temp = feature_all.drop(temp_index,axis=0)
temp.to_csv(f'{feature_path}/feature_laser.csv')

# X線CT用のfeature.csv作成
temp = feature_all.loc[temp_index,:]
temp.to_csv(f'{feature_path}/feature_XrayCT.csv')

['feature_01.csv', 'feature_02.csv', 'feature_03.csv', 'feature_04.csv', 'feature_06.csv', 'feature_07.csv', 'feature_08.csv', 'feature_09.csv', 'feature_10.csv', 'feature_11.csv', 'feature_12.csv']
CPU times: user 103 ms, sys: 7.29 ms, total: 111 ms
Wall time: 111 ms
